<a href="https://colab.research.google.com/github/leahandofir/earthformer-inference-experiments/blob/main/earthformer_inference_experiment_without_conda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Genral:
**The goal** - try to understand what we need to do to run model(x) and get a result. **STILL DID NOT REACHED THAT GOAL**

**The structure of the code** - 
- Train
- Test
- The architecture of the model - the model recieves an example x and returns prediction y. The model has a function named "eval" to turn off behaviors that are relevent in the training process (like dropout and batch normalization), this is realy important to do when inferancing!
- Data set - the code that responsible for assamble the raw samples as python data structures. This is a class that usually inhirites from a DataSet class of pytorch. Has __getItem__ method that recieves an index and returns a sample x with that index. Also has a __len__ methos that returns the number of samples in the data set.
- Data Loader - wraps the Data set. Has a certain number of workers to concurrantly and efficiently read the data set. Has a queue that always prepare the next mini batch so the usage of the GPU will be efficient. 
collate_fn - a function that aggragates a batch of samples from the data set loader in a data structure that the model can train on.
comment - we want the batch to be as large as possible to has less transitions from the CPU to the GPU.

Solving error 21/4/2023:

Tried to solve the error `ImportError: cannot import name '_PATH' from 'pytorch_lightning.utilities.types`
with some google advices with no success.
after that saw this error in the pip outputs:
```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
```
also this:
```
ERROR: pip's dependency resolver does not currently 
take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
```
so decided to try and install everything with cuda 11.8 and let pip decide the versions.
after that it just worked.
general good things to copy from their code:
- they use os.path a lot. checking if directories exist, and if not they create them. we need our code to be as little as possible dependent on the machine.
- they have utils directory of all the general needed things (like download a given url)



Dependencies

In [ ]:
%%shell
# install dependencies
pip install --upgrade pip
python3 -m pip install torch==2.0.0+cu118 torchvision -f https://download.pytorch.org/whl/torch_stable.html
python3 -m pip install "pytorch_lightning>=1.6.4,<1.8.0"
python3 -m pip install xarray netcdf4 opencv-python
git clone https://github.com/amazon-science/earth-forecasting-transformer.git
cd earth-forecasting-transformer
python3 -m pip install -U -e . --no-build-isolation
pip install -v --disable-pip-version-check --no-cache-dir pytorch-extension git+https://github.com/NVIDIA/apex.git

In [ ]:
%%shell
# install aws cli to check out sevir dataset
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install

In [22]:
%%shell
# checking the aws cli installation and the size of the sevir dataset
aws --version

aws-cli/1.27.125 Python/3.10.11 Linux/5.10.147+ botocore/1.29.125


In [12]:
%cd /content/earth-forecasting-transformer/src

/content/earth-forecasting-transformer/src


The code is from tests/unittests/test_pretrained_checkpoints.py:

In [13]:
import warnings
import os
from omegaconf import OmegaConf
import pytest
import numpy as np
import torch
from torch.nn import functional as F
import torchmetrics
from einops import rearrange
from earthformer.config import cfg
from earthformer.utils.checkpoint import s3_download_pretrained_ckpt
from earthformer.utils.layout import layout_to_in_out_slice
from earthformer.utils.utils import download
from earthformer.cuboid_transformer.cuboid_transformer import CuboidTransformerModel
from earthformer.cuboid_transformer.cuboid_transformer_unet_dec import CuboidTransformerAuxModel


NUM_TEST_ITER = 16  # max = 32 since saved `unittest_data.pt` only contains the first 0 to 31 data entries.
test_data_dir = os.path.join(cfg.root_dir, "tests", "unittests", "test_pretrained_checkpoints_data")

def s3_download_unittest_data(data_name):
    test_data_path = os.path.join(test_data_dir, data_name)
    if not os.path.exists(test_data_path):
        os.makedirs(test_data_dir, exist_ok=True)
        download(url=f"s3://deep-earth/experiments/earthformer/unittests/{data_name}", path=test_data_path)


def config_cuboid_transformer(cfg, model_type="CuboidTransformerModel"):
    model_cfg = OmegaConf.to_object(cfg.model)
    num_blocks = len(model_cfg["enc_depth"])
    if isinstance(model_cfg["self_pattern"], str):
        enc_attn_patterns = [model_cfg.pop("self_pattern")] * num_blocks
    else:
        enc_attn_patterns = OmegaConf.to_container(model_cfg.pop("self_pattern"))
    model_cfg["enc_attn_patterns"] = enc_attn_patterns
    if isinstance(model_cfg["cross_self_pattern"], str):
        dec_self_attn_patterns = [model_cfg.pop("cross_self_pattern")] * num_blocks
    else:
        dec_self_attn_patterns = OmegaConf.to_container(model_cfg.pop("cross_self_pattern"))
    model_cfg["dec_self_attn_patterns"] = dec_self_attn_patterns
    if isinstance(model_cfg["cross_pattern"], str):
        dec_cross_attn_patterns = [model_cfg.pop("cross_pattern")] * num_blocks
    else:
        dec_cross_attn_patterns = OmegaConf.to_container(model_cfg.pop("cross_pattern"))
    model_cfg["dec_cross_attn_patterns"] = dec_cross_attn_patterns
    if model_type == "CuboidTransformerModel":
        model = CuboidTransformerModel(**model_cfg)
    elif model_type == "CuboidTransformerAuxModel":
        model = CuboidTransformerAuxModel(**model_cfg)
    else:
        raise ValueError(f"Invalid model_type {model_type}. Must be 'CuboidTransformerModel' or ''.")
    return model

def test_sevir():
    pretrained_ckpt_name = "earthformer_sevir.pt"
    test_data_name = "unittest_sevir_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "sevir", "earthformer_sevir_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerModel").to(device)
    model.eval()
    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on SEVIR test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0
    with torch.no_grad():
        for batch in test_data:
            data_seq = batch['vil'].contiguous().to(device)
            x = data_seq[in_slice]
            y = data_seq[out_slice]
            y_hat = model(x)
            test_mse_metrics(y_hat, y)
            test_mae_metrics(y_hat, y)
            counter += 1
            if counter >= NUM_TEST_ITER:
                break
    test_mse = test_mse_metrics.compute()
    test_mae = test_mae_metrics.compute()
    assert test_mse < 1E-2
    assert test_mae < 5E-2

def test_enso():
    pretrained_ckpt_name = "earthformer_icarenso2021.pt"
    test_data_name = "unittest_icarenso2021_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "enso", "earthformer_enso_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerModel").to(device)
    model.eval()

    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on ENSO test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0
    with torch.no_grad():
        for batch in test_data:
            sst_seq, nino_target = batch
            data_seq = sst_seq.float().unsqueeze(-1).to(device)
            x = data_seq[in_slice]
            y = data_seq[out_slice]
            y_hat = model(x)
            test_mse_metrics(y_hat, y)
            test_mae_metrics(y_hat, y)
            counter += 1
            if counter >= NUM_TEST_ITER:
                break
    test_mse = test_mse_metrics.compute()
    test_mae = test_mae_metrics.compute()
    assert test_mse < 5E-4
    assert test_mae < 2E-2

def test_earthnet():
    data_channels = 4
    pretrained_ckpt_name = "earthformer_earthnet2021.pt"
    test_data_name = "unittest_earthnet2021_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "earthnet_w_meso", "earthformer_earthnet_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerAuxModel").to(device)
    model.eval()

    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on EarthNet2021 test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0
    with torch.no_grad():
        for batch in test_data:
            highresdynamic = batch["highresdynamic"].to(device)
            seq = highresdynamic[..., :data_channels]
            print(seq.shape)
            # mask from dataloader: 1 for mask and 0 for non-masked
            mask = highresdynamic[..., data_channels: data_channels + 1][out_slice]
            in_seq = seq[in_slice]
            target_seq = seq[out_slice]
            # process aux data
            highresstatic = batch["highresstatic"].to(device)  # (b c h w)
            mesodynamic = batch["mesodynamic"].to(device)  # (b t h w c)
            mesostatic = batch["mesostatic"].to(device)  # (b c h w)
            mesodynamic_interp = rearrange(mesodynamic,
                                           "b t h w c -> b c t h w")
            mesodynamic_interp = F.interpolate(mesodynamic_interp,
                                               size=(layout_cfg.in_len + layout_cfg.out_len,
                                                     layout_cfg.img_height,
                                                     layout_cfg.img_width),
                                               mode="nearest")
            highresstatic_interp = rearrange(highresstatic,
                                             "b c h w -> b c 1 h w")
            highresstatic_interp = F.interpolate(highresstatic_interp,
                                                 size=(layout_cfg.in_len + layout_cfg.out_len,
                                                       layout_cfg.img_height,
                                                       layout_cfg.img_width),
                                                 mode="nearest")
            mesostatic_interp = rearrange(mesostatic,
                                          "b c h w -> b c 1 h w")
            mesostatic_interp = F.interpolate(mesostatic_interp,
                                              size=(layout_cfg.in_len + layout_cfg.out_len,
                                                    layout_cfg.img_height,
                                                    layout_cfg.img_width),
                                              mode="nearest")
            aux_data = torch.cat((highresstatic_interp, mesodynamic_interp, mesostatic_interp),
                                 dim=1)
            aux_data = rearrange(aux_data,
                                 "b c t h w -> b t h w c")
            pred_seq = model(in_seq, aux_data[in_slice], aux_data[out_slice])
            test_mse_metrics(pred_seq * (1 - mask), target_seq * (1 - mask))
            test_mae_metrics(pred_seq * (1 - mask), target_seq * (1 - mask))
            counter += 1
            if counter >= NUM_TEST_ITER:
                break
    test_mse = test_mse_metrics.compute()
    test_mae = test_mae_metrics.compute()
    assert test_mse < 5E-4
    assert test_mae < 1E-2

In [ ]:
  # trying to run some parts to understand what is going on
    pretrained_ckpt_name = "earthformer_sevir.pt"
    test_data_name = "unittest_sevir_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "sevir", "earthformer_sevir_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerModel").to(device)
    model.eval()

In [ ]:
model # holds all layers of the model

In [ ]:
    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on SEVIR test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0

In [ ]:
layout_cfg

{'in_len': 13, 'out_len': 12, 'layout': 'NTHWC'}

In [ ]:
test_data # this is data!!! :O how can we see the pictures?!

Inspecting RAW sevir data set

In [23]:
# looking at the files... I want to download a small amount and try the inferance on them.
!aws s3 ls --no-sign-request s3://sevir/data/vil --recursive --human-readable

2020-04-15 02:49:50    9.7 GiB data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0501_0831.h5
2020-04-15 02:49:50   12.8 GiB data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0901_1231.h5
2020-04-15 02:49:57    1.3 GiB data/vil/2017/SEVIR_VIL_STORMEVENTS_2017_0101_0630.h5
2020-04-15 02:50:00    4.6 GiB data/vil/2017/SEVIR_VIL_STORMEVENTS_2017_0701_1231.h5
2020-04-15 02:50:09   11.5 GiB data/vil/2018/SEVIR_VIL_RANDOMEVENTS_2018_0101_0430.h5
2020-04-15 02:52:12   15.9 GiB data/vil/2018/SEVIR_VIL_RANDOMEVENTS_2018_0501_0831.h5
2020-04-15 02:58:05   15.1 GiB data/vil/2018/SEVIR_VIL_RANDOMEVENTS_2018_0901_1231.h5
2020-04-15 03:06:49    5.3 GiB data/vil/2018/SEVIR_VIL_STORMEVENTS_2018_0101_0630.h5
2020-04-15 03:10:18    5.7 GiB data/vil/2018/SEVIR_VIL_STORMEVENTS_2018_0701_1231.h5
2020-04-15 18:28:01   15.2 GiB data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5
2020-04-15 18:28:02   15.6 GiB data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5
2020-04-15 18:28:02   15.1 GiB data/vil/2019/SEVIR_VIL_RAN

In [24]:
# lets download only one of the h5 files
!aws s3 cp --no-sign-request --recursive s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: the following arguments are required: paths


In [20]:
import os
from typing import Union, Dict, Sequence, Tuple, List
import numpy as np
import datetime
import pandas as pd
import torch
from torch.utils.data import Dataset as TorchDataset, DataLoader
from pytorch_lightning import LightningDataModule
from ...config import cfg
from .sevir_dataloader import SEVIRDataLoader


class SEVIRTorchDataset(TorchDataset):

    def __init__(self,
                 seq_len: int = 25,
                 raw_seq_len: int = 49,
                 sample_mode: str = "sequent",
                 stride: int = 12,
                 batch_size: int = 1,
                 layout: str = "NHWT",
                 num_shard: int = 1,
                 rank: int = 0,
                 split_mode: str = "uneven",
                 sevir_catalog: Union[str, pd.DataFrame] = None,
                 sevir_data_dir: str = None,
                 start_date: datetime.datetime = None,
                 end_date: datetime.datetime = None,
                 datetime_filter = None,
                 catalog_filter = "default",
                 shuffle: bool = False,
                 shuffle_seed: int = 1,
                 output_type = np.float32,
                 preprocess: bool = True,
                 rescale_method: str = "01",
                 verbose: bool = False):
        super(SEVIRTorchDataset, self).__init__()
        self.layout = layout
        self.sevir_dataloader = SEVIRDataLoader(
            data_types=["vil", ],
            seq_len=seq_len,
            raw_seq_len=raw_seq_len,
            sample_mode=sample_mode,
            stride=stride,
            batch_size=batch_size,
            layout=layout,
            num_shard=num_shard,
            rank=rank,
            split_mode=split_mode,
            sevir_catalog=sevir_catalog,
            sevir_data_dir=sevir_data_dir,
            start_date=start_date,
            end_date=end_date,
            datetime_filter=datetime_filter,
            catalog_filter=catalog_filter,
            shuffle=shuffle,
            shuffle_seed=shuffle_seed,
            output_type=output_type,
            preprocess=preprocess,
            rescale_method=rescale_method,
            downsample_dict=None,
            verbose=verbose)

    def __getitem__(self, index):
        data_dict = self.sevir_dataloader._idx_sample(index=index)
        return data_dict

    def __len__(self):
        return self.sevir_dataloader.__len__()

    def collate_fn(self, data_dict_list):
        r"""
        Parameters
        ----------
        data_dict_list:  list[Dict[str, torch.Tensor]]
        Returns
        -------
        merged_data: Dict[str, torch.Tensor]
            batch_size = len(data_dict_list) * data_dict["key"].batch_size
        """
        batch_dim = self.layout.find('N')
        data_list_dict = {
            key: [data_dict[key]
                  for data_dict in data_dict_list]
            for key in data_dict_list[0]}
        # TODO: key "mask" is not handled. Temporally fine since this func is not used
        data_list_dict.pop("mask", None)
        merged_dict = {
            key: torch.cat(data_list,
                           dim=batch_dim)
            for key, data_list in data_list_dict.items()}
        merged_dict["mask"] = None
        return merged_dict

    def get_torch_dataloader(self,
                             outer_batch_size=1,
                             collate_fn=None,
                             num_workers=1):
        # TODO: num_workers > 1
        r"""
        We set the batch_size in Dataset by default, so outer_batch_size should be 1.
        In this case, not using `collate_fn` can save time.
        """
        if outer_batch_size == 1:
            collate_fn = lambda x:x[0]
        else:
            if collate_fn is None:
                collate_fn = self.collate_fn
        dataloader = DataLoader(
            dataset=self,
            batch_size=outer_batch_size,
            collate_fn=collate_fn,
            num_workers=num_workers)
        return dataloader


def check_aws():
    r"""
    Check if aws cli is installed.
    """
    if os.system("which aws") != 0:
        raise RuntimeError("AWS CLI is not installed! Please install it first. See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html")


def download_SEVIR(save_dir=None):
    r"""
    Downloaded dataset is saved in save_dir/sevir
    """

    check_aws()

    if save_dir is None:
        save_dir = cfg.datasets_dir
    sevir_dir = os.path.join(save_dir, "sevir")
    if os.path.exists(sevir_dir):
        raise FileExistsError(f"Path to save SEVIR dataset {sevir_dir} already exists!")
    else:
        os.makedirs(sevir_dir)
        os.system(f"aws s3 cp --no-sign-request s3://sevir/CATALOG.csv "
                  f"{os.path.join(sevir_dir, 'CATALOG.csv')}")
        os.system(f"aws s3 cp --no-sign-request --recursive s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5 "
                  f"{os.path.join(sevir_dir, 'data', 'vil')}")

class SEVIRLightningDataModule(LightningDataModule):

    def __init__(self,
                 seq_len: int = 25,
                 sample_mode: str = "sequent",
                 stride: int = 12,
                 batch_size: int = 1,
                 layout: str = "NHWT",
                 output_type = np.float32,
                 preprocess: bool = True,
                 rescale_method: str = "01",
                 verbose: bool = False,
                 # datamodule_only
                 dataset_name: str = "sevir",
                 start_date: Tuple[int] = None,
                 train_val_split_date: Tuple[int] = (2019, 1, 1),
                 train_test_split_date: Tuple[int] = (2019, 6, 1),
                 end_date: Tuple[int] = None,
                 num_workers: int = 1,
                 ):
        super(SEVIRLightningDataModule, self).__init__()
        self.seq_len = seq_len
        self.sample_mode = sample_mode
        self.stride = stride
        self.batch_size = batch_size
        self.layout = layout
        self.output_type = output_type
        self.preprocess = preprocess
        self.rescale_method = rescale_method
        self.verbose = verbose
        self.num_workers = num_workers
        if dataset_name == "sevir":
            sevir_root_dir = os.path.join(cfg.datasets_dir, "sevir")
            catalog_path = os.path.join(sevir_root_dir, "CATALOG.csv")
            raw_data_dir = os.path.join(sevir_root_dir, "data")
            raw_seq_len = 49
            interval_real_time = 5
            img_height = 384
            img_width = 384
        elif dataset_name == "sevir_lr":
            sevir_root_dir = os.path.join(cfg.datasets_dir, "sevir_lr")
            catalog_path = os.path.join(sevir_root_dir, "CATALOG.csv")
            raw_data_dir = os.path.join(sevir_root_dir, "data")
            raw_seq_len = 25
            interval_real_time = 10
            img_height = 128
            img_width = 128
        else:
            raise ValueError(f"Wrong dataset name {dataset_name}. Must be 'sevir' or 'sevir_lr'.")
        self.dataset_name = dataset_name
        self.sevir_root_dir = sevir_root_dir
        self.catalog_path = catalog_path
        self.raw_data_dir = raw_data_dir
        self.raw_seq_len = raw_seq_len
        self.interval_real_time = interval_real_time
        self.img_height = img_height
        self.img_width = img_width
        # train val test split
        self.start_date = datetime.datetime(*start_date) \
            if start_date is not None else None
        self.train_val_split_date = datetime.datetime(*train_val_split_date)
        self.train_test_split_date = datetime.datetime(*train_test_split_date)
        self.end_date = datetime.datetime(*end_date) \
            if end_date is not None else None

    def prepare_data(self) -> None:
        if os.path.exists(self.sevir_root_dir):
            # Further check
            assert os.path.exists(self.catalog_path), f"CATALOG.csv not found! Should be located at {self.catalog_path}"
            assert os.path.exists(self.raw_data_dir), f"SEVIR data not found! Should be located at {self.raw_data_dir}"
        else:
            if self.dataset_name == "sevir":
                download_SEVIR()
            else:  # "sevir_lr"
                raise NotImplementedError

    def setup(self, stage = None) -> None:
        self.sevir_train = SEVIRTorchDataset(
            sevir_catalog=self.catalog_path,
            sevir_data_dir=self.raw_data_dir,
            raw_seq_len=self.raw_seq_len,
            split_mode="uneven",
            shuffle=True,
            seq_len=self.seq_len,
            stride=self.stride,
            sample_mode=self.sample_mode,
            batch_size=self.batch_size,
            layout=self.layout,
            num_shard=1, rank=0,
            start_date=self.start_date,
            end_date=self.train_val_split_date,
            output_type=self.output_type,
            preprocess=self.preprocess,
            rescale_method=self.rescale_method,
            verbose=self.verbose,)
        self.sevir_val = SEVIRTorchDataset(
            sevir_catalog=self.catalog_path,
            sevir_data_dir=self.raw_data_dir,
            raw_seq_len=self.raw_seq_len,
            split_mode="uneven",
            shuffle=False,
            seq_len=self.seq_len,
            stride=self.stride,
            sample_mode=self.sample_mode,
            batch_size=self.batch_size,
            layout=self.layout,
            num_shard=1, rank=0,
            start_date=self.train_val_split_date,
            end_date=self.train_test_split_date,
            output_type=self.output_type,
            preprocess=self.preprocess,
            rescale_method=self.rescale_method,
            verbose=self.verbose, )
        self.sevir_test = SEVIRTorchDataset(
            sevir_catalog=self.catalog_path,
            sevir_data_dir=self.raw_data_dir,
            raw_seq_len=self.raw_seq_len,
            split_mode="uneven",
            shuffle=False,
            seq_len=self.seq_len,
            stride=self.stride,
            sample_mode=self.sample_mode,
            batch_size=self.batch_size,
            layout=self.layout,
            num_shard=1, rank=0,
            start_date=self.train_test_split_date,
            end_date=self.end_date,
            output_type=self.output_type,
            preprocess=self.preprocess,
            rescale_method=self.rescale_method,
            verbose=self.verbose,)
        self.sevir_predict = SEVIRTorchDataset(
            sevir_catalog=self.catalog_path,
            sevir_data_dir=self.raw_data_dir,
            raw_seq_len=self.raw_seq_len,
            split_mode="uneven",
            shuffle=False,
            seq_len=self.seq_len,
            stride=self.stride,
            sample_mode=self.sample_mode,
            batch_size=self.batch_size,
            layout=self.layout,
            num_shard=1, rank=0,
            start_date=self.train_test_split_date,
            end_date=self.end_date,
            output_type=self.output_type,
            preprocess=self.preprocess,
            rescale_method=self.rescale_method,
            verbose=self.verbose,)
        
    def train_dataloader(self):
        return self.sevir_train.get_torch_dataloader(num_workers=self.num_workers)

    def val_dataloader(self):
        return self.sevir_val.get_torch_dataloader(num_workers=self.num_workers)

    def test_dataloader(self):
        return self.sevir_test.get_torch_dataloader(num_workers=self.num_workers)

    def predict_dataloader(self):
        return self.sevir_predict.get_torch_dataloader(num_workers=self.num_workers)

    @property
    def num_train_samples(self):
        return len(self.sevir_train)

    @property
    def num_val_samples(self):
        return len(self.sevir_val)

    @property
    def num_test_samples(self):
        return len(self.sevir_test)

ImportError: ignored

# New section